# **TP2**

##*Organización de Datos (75.06/95.58) - Primer Cuatrimestre 2021*

###### Julio de 2021
---

In [ ]:
# Importamos librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# Importamos los archivos de entrada desde google drive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1d3bvCoox6nxGIWx81EMEQl_hJ1Y0Jp1q/view?usp=sharing' 
  
# to get the id part of the file
id = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train_values.csv')

train_values=pd.read_csv('train_values.csv', index_col='building_id')

In [ ]:
link = 'https://drive.google.com/file/d/1zsOf_CScGSl6O2CBbgQUX7TH3CALpXbl/view?usp=sharing' 
  
# to get the id part of the file
id = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train_labels.csv')

train_labels=pd.read_csv('train_labels.csv', index_col='building_id')

#Damage grade:
#1 - Low damage
#2 - Medium damage
#3 - Serious damage


In [ ]:
link = 'https://drive.google.com/file/d/1kJo-J5HVYul6A5nC7xMlly9JyHPD_OcY/view?usp=sharing' 
  
# to get the id part of the file
id = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test_values.csv')

test_values=pd.read_csv('test_values.csv', index_col='building_id')

In [ ]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler

# the model
from sklearn.ensemble import RandomForestClassifier

# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV

#metric
from sklearn.metrics import f1_score

In [ ]:
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
cat_cols=['land_surface_condition','foundation_type','roof_type','ground_floor_type','other_floor_type',
          'position','plan_configuration','legal_ownership_status']
for cc in cat_cols:
    train_values[cc]=train_values[cc].astype('category')
    test_values[cc]=test_values[cc].astype('category')

train_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 38 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   geo_level_1_id                          260601 non-null  int64   
 1   geo_level_2_id                          260601 non-null  int64   
 2   geo_level_3_id                          260601 non-null  int64   
 3   count_floors_pre_eq                     260601 non-null  int64   
 4   age                                     260601 non-null  int64   
 5   area_percentage                         260601 non-null  int64   
 6   height_percentage                       260601 non-null  int64   
 7   land_surface_condition                  260601 non-null  category
 8   foundation_type                         260601 non-null  category
 9   roof_type                               260601 non-null  category
 10  ground_floor_type          

In [ ]:
X_dev = train_values.drop(columns=['geo_level_2_id','geo_level_3_id'])
X_test = test_values.drop(columns=['geo_level_2_id','geo_level_3_id'])

Y_dev = train_labels


print(X_dev.info())
print('\n')
print(X_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 36 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   geo_level_1_id                          260601 non-null  int64   
 1   count_floors_pre_eq                     260601 non-null  int64   
 2   age                                     260601 non-null  int64   
 3   area_percentage                         260601 non-null  int64   
 4   height_percentage                       260601 non-null  int64   
 5   land_surface_condition                  260601 non-null  category
 6   foundation_type                         260601 non-null  category
 7   roof_type                               260601 non-null  category
 8   ground_floor_type                       260601 non-null  category
 9   other_floor_type                        260601 non-null  category
 10  position                   

# Encodeamos *geo_level_1_id* en 3 columnas ($E_{\mathrm{frac}}$)



In [ ]:
df_0=X_dev.merge(Y_dev, left_index=True, right_index=True, how='inner')
df=df_0[['damage_grade','geo_level_1_id']].copy().set_index(['damage_grade','geo_level_1_id'])
df['countingindex']=1
df=df.groupby(['damage_grade','geo_level_1_id']).agg({'countingindex':'sum'}).unstack().droplevel(0,axis=1).fillna(0)
df

geo_level_1_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
damage_grade,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,337.0,411.0,85.0,245.0,521.0,446.0,2108.0,1033.0,654.0,561.0,1211.0,386.0,199.0,1966.0,191.0,163.0,200.0,285.0,72.0,43.0,3311.0,322.0,811.0,72.0,270.0,468.0,8028.0,465.0,0.0,8.0,252.0
2,3075.0,1985.0,610.0,4550.0,11164.0,2014.0,16222.0,11273.0,8513.0,2733.0,12107.0,4672.0,2310.0,6275.0,1247.0,1673.0,3188.0,3913.0,786.0,263.0,11860.0,5857.0,4624.0,768.0,908.0,4384.0,12645.0,6007.0,157.0,349.0,2127.0
3,599.0,305.0,236.0,2745.0,2883.0,230.0,6051.0,6688.0,9913.0,664.0,8761.0,3162.0,685.0,1367.0,276.0,484.0,944.0,17615.0,2331.0,66.0,2045.0,8710.0,817.0,281.0,132.0,772.0,1942.0,6060.0,108.0,39.0,307.0


In [ ]:
frac_gl1=np.divide(np.array(df.iloc[0]),np.array(df.sum(axis=0)))
frac_gl1=pd.concat([pd.Series(frac_gl1), pd.Series(np.divide(np.array(df.iloc[1]),np.array(df.sum(axis=0))))],axis=1,keys=['1','2'])
frac_gl1=pd.concat([frac_gl1, pd.Series(np.divide(np.array(df.iloc[2]),np.array(df.sum(axis=0))))],axis=1)
frac_gl1=frac_gl1.rename(columns={0:'3'})
frac_gl1

,1,2,3
0,0.084019,0.766642,0.149339
1,0.152166,0.734913,0.112921
2,0.091300,0.655209,0.253491
3,0.032493,0.603448,0.364058
4,0.035763,0.766337,0.197900
5,0.165799,0.748699,0.085502
6,0.086461,0.665354,0.248185
7,0.054386,0.593503,0.352111
8,0.034277,0.446174,0.519549
9,0.141738,0.690500,0.167761


In [ ]:
print(X_dev.shape)
X_dev_enc=X_dev.merge(frac_gl1,how='left',left_on='geo_level_1_id',right_index=True).rename(columns={'1':'Frac_damage_1','2':'Frac_damage_2','3':'Frac_damage_3'})
X_dev_enc=X_dev_enc.drop(columns='geo_level_1_id')
print(X_dev_enc.shape)
X_dev_enc=pd.get_dummies(X_dev_enc)
print(X_dev_enc.shape)
X_dev_enc.info()

(260601, 36)
(260601, 38)
(260601, 68)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 68 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   count_floors_pre_eq                     260601 non-null  int64  
 1   age                                     260601 non-null  int64  
 2   area_percentage                         260601 non-null  int64  
 3   height_percentage                       260601 non-null  int64  
 4   has_superstructure_adobe_mud            260601 non-null  int64  
 5   has_superstructure_mud_mortar_stone     260601 non-null  int64  
 6   has_superstructure_stone_flag           260601 non-null  int64  
 7   has_superstructure_cement_mortar_stone  260601 non-null  int64  
 8   has_superstructure_mud_mortar_brick     260601 non-null  int64  
 9   has_superstructure_cement_mortar_brick  260601 non-null  int64  
 10  

In [ ]:
print(X_test.shape)
X_test_enc=X_test.merge(frac_gl1,how='left',left_on='geo_level_1_id',right_index=True).rename(columns={'1':'Frac_damage_1','2':'Frac_damage_2','3':'Frac_damage_3'})
X_test_enc=X_test_enc.drop(columns='geo_level_1_id')
print(X_test_enc.shape)
X_test_enc=pd.get_dummies(X_test_enc)
print(X_test_enc.shape)
X_test_enc.info()

(86868, 36)
(86868, 38)
(86868, 68)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 86868 entries, 300051 to 501372
Data columns (total 68 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   count_floors_pre_eq                     86868 non-null  int64  
 1   age                                     86868 non-null  int64  
 2   area_percentage                         86868 non-null  int64  
 3   height_percentage                       86868 non-null  int64  
 4   has_superstructure_adobe_mud            86868 non-null  int64  
 5   has_superstructure_mud_mortar_stone     86868 non-null  int64  
 6   has_superstructure_stone_flag           86868 non-null  int64  
 7   has_superstructure_cement_mortar_stone  86868 non-null  int64  
 8   has_superstructure_mud_mortar_brick     86868 non-null  int64  
 9   has_superstructure_cement_mortar_brick  86868 non-null  int64  
 10  has_superstructu

# Modelo: RF

In [ ]:
model = RandomForestClassifier(max_depth=20,min_samples_split=12,n_estimators=300,criterion='gini',random_state=2)

## Set S3

In [ ]:
X_dev_enc['Height_Area']=X_dev_enc['height_percentage']/X_dev_enc['area_percentage']
X_test_enc['Height_Area']=X_test_enc['height_percentage']/X_test_enc['area_percentage']

print(X_dev_enc.info())
print('\n')
print(X_test_enc.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 69 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   count_floors_pre_eq                     260601 non-null  int64  
 1   age                                     260601 non-null  int64  
 2   area_percentage                         260601 non-null  int64  
 3   height_percentage                       260601 non-null  int64  
 4   has_superstructure_adobe_mud            260601 non-null  int64  
 5   has_superstructure_mud_mortar_stone     260601 non-null  int64  
 6   has_superstructure_stone_flag           260601 non-null  int64  
 7   has_superstructure_cement_mortar_stone  260601 non-null  int64  
 8   has_superstructure_mud_mortar_brick     260601 non-null  int64  
 9   has_superstructure_cement_mortar_brick  260601 non-null  int64  
 10  has_superstructure_timber              

In [ ]:
rf_clf=model.fit(X_dev_enc, Y_dev.values.ravel())

# Para subir

## Make Predictions

In [ ]:
Y_pred = rf_clf.predict(X_test_enc)
predictions = Y_pred 

## Save Submission

In [ ]:
link = 'https://drive.google.com/file/d/1mFzu1Am6IMhxp1zLvTLaGu0Az-VkIril/view?usp=sharing' 
  
# to get the id part of the file
id = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('submission_format.csv')

submission_format=pd.read_csv('submission_format.csv', index_col='building_id')

In [ ]:
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [ ]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [ ]:
my_submission.to_csv('submission.csv')